In [1]:
import requests as rq
import pandas as pd
from pprint import pprint as pp

In [ ]:
#there's a python package (pigshare): https://pypi.org/project/pigshare/
#hasn't been updated since 2016, will need to check if still works accurately
#pip install pigshare errored out
#if did work, along lines of: pigshare search_articles --search_term [search_term]

In [ ]:
#Looks like if can get all article ids, can get full metadata by article id
#will need to search: articles, collections, projects

In [ ]:
#curl syntax: #curl https://api.figshare.com/v2?access_token=ACCESS_TOKEN

In [2]:
#there has to be a better way to load a variable from a .py file
exec(open("Figshare_access_token.py").read())

In [4]:
#code cribbed from: https://gist.github.com/jezcope/e01bc2e85adbba9d8981

HEADERS = {'Authorization': 'token '+TOKEN}

r = rq.post('https://api.figshare.com/v2/articles/search', params={'search_for': 'machine learning'}, headers=HEADERS)
print(r.status_code)
results = r.json()
pp(results)

#this works but I don't understand it enough to be confident about HOW it works...
#this also must be paginated results, so how do we specify page number?

200
[{'defined_type': 8,
  'defined_type_name': 'thesis',
  'doi': '10.1184/r1/6714890.v1',
  'group_id': 18614,
  'handle': '',
  'id': 6714890,
  'published_date': '2012-08-01T00:00:00Z',
  'resource_doi': '',
  'resource_title': '',
  'thumb': 'https://s3-eu-west-1.amazonaws.com/ppreviews-cmu-49810598590254/12247874/thumb.png',
  'timeline': {'firstOnline': '2018-06-30T23:16:04',
               'posted': '2012-08-01T00:00:00',
               'revision': '2018-06-30T23:16:05'},
  'title': 'Attribute Learning using Joint Human and Machine Computation',
  'url': 'https://api.figshare.com/v2/articles/6714890',
  'url_private_api': 'https://api.figshare.com/v2/account/articles/6714890',
  'url_private_html': 'https://figshare.com/account/articles/6714890',
  'url_public_api': 'https://api.figshare.com/v2/articles/6714890',
  'url_public_html': 'https://kilthub.cmu.edu/articles/thesis/Attribute_Learning_using_Joint_Human_and_Machine_Computation/6714890'},
 {'defined_type': 8,
  'defined_t